In [22]:
import joblib

X_train = joblib.load('Test Kompas\X_train_clean.pkl')
X_valid = joblib.load('Test Kompas\X_valid_clean.pkl')
X_test = joblib.load('Test Kompas\X_test_clean.pkl')

y_train = joblib.load('Test Kompas\y_train.pkl')
y_valid = joblib.load('Test Kompas\y_valid.pkl')
y_test = joblib.load('Test Kompas\y_test.pkl')

In [23]:
X_train = X_train[['FullText']]
X_valid = X_valid[['FullText']]
X_test = X_test[['FullText']]

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers

# Tentukan ukuran vocabulary dan panjang sequence maksimum
vocab_size = 5000  # Sesuaikan dengan data Anda
sequence_length = 300  # Sesuaikan dengan panjang rata-rata teks Anda

# Buat layer TextVectorization
vectorize_layer = TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Lakukan adaptasi pada data pelatihan untuk membuat vocabulary
text_ds = tf.data.Dataset.from_tensor_slices(X_train['FullText']).batch(128)
vectorize_layer.adapt(text_ds)

# Buat model sequential
model = tf.keras.Sequential([
    vectorize_layer,  # Layer vektorisasi
    layers.Embedding(vocab_size, 128, mask_zero=True),  # Layer embedding
    layers.Bidirectional(layers.LSTM(64)),  # Layer LSTM dua arah
    layers.Dense(64, activation='relu'),  # Layer dense
    layers.Dense(len(np.unique(y_train)), activation='softmax')  # Output layer dengan softmax
])

# Kompilasi model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Inisialisasi LabelEncoder
le = LabelEncoder()

# Fit dan transform label menjadi nilai numerik
train_labels = le.fit_transform(y_train)
valid_labels = le.transform(y_valid)

# Konversi data teks dan label menjadi dataset TensorFlow
train_ds = tf.data.Dataset.from_tensor_slices((X_train['FullText'], train_labels)).batch(32)
valid_ds = tf.data.Dataset.from_tensor_slices((X_valid['FullText'], valid_labels)).batch(32)

# Latih model
history = model.fit(train_ds, validation_data=valid_ds, epochs=15)

Epoch 1/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 181ms/step - accuracy: 0.2047 - loss: 2.9683 - val_accuracy: 0.2550 - val_loss: 2.6153
Epoch 2/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 161ms/step - accuracy: 0.2668 - loss: 2.3618 - val_accuracy: 0.3050 - val_loss: 2.1932
Epoch 3/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 165ms/step - accuracy: 0.3657 - loss: 2.0061 - val_accuracy: 0.3850 - val_loss: 2.1982
Epoch 4/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 160ms/step - accuracy: 0.5040 - loss: 1.7512 - val_accuracy: 0.4550 - val_loss: 1.9647
Epoch 5/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 158ms/step - accuracy: 0.5975 - loss: 1.3594 - val_accuracy: 0.5125 - val_loss: 1.7332
Epoch 6/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 159ms/step - accuracy: 0.7000 - loss: 1.0382 - val_accuracy: 0.5475 - val_loss: 1.7349
Epoch 7/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 158ms/step - accuracy: 0.7632 - loss: 0.8050 - val_accuracy: 0.5050 - val_loss: 1.9496
Epoch 8/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 158ms/step - accuracy: 0.7667 - loss: 0.7672 - val_accuracy: 0

In [7]:
# Evaluasi model
loss, accuracy = model.evaluate(valid_ds)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.6116 - loss: 1.9987
Loss: 2.2408995628356934
Accuracy: 0.5799999833106995


In [24]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers

# Tentukan ukuran vocabulary dan panjang sequence maksimum
vocab_size = 5000  # Sesuaikan dengan data Anda
sequence_length = 300  # Sesuaikan dengan panjang rata-rata teks Anda

# Buat layer TextVectorization
vectorize_layer = TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Lakukan adaptasi pada data pelatihan untuk membuat vocabulary
text_ds = tf.data.Dataset.from_tensor_slices(X_train['FullText']).batch(128)
vectorize_layer.adapt(text_ds)

# Buat model sequential
model = tf.keras.Sequential([
    vectorize_layer,  # Layer vektorisasi
    layers.Embedding(vocab_size, 128, mask_zero=True),  # Layer embedding
    layers.Bidirectional(layers.LSTM(64, return_sequences=True)),  # Layer LSTM dua arah
    layers.Dropout(0.5),  # Tambahkan lapisan dropout
    layers.Bidirectional(layers.LSTM(64)),  # Layer LSTM dua arah
    layers.Dense(64, activation='relu'),  # Layer dense
    layers.Dense(len(np.unique(y_train)), activation='softmax')  # Output layer dengan softmax
])

# Kompilasi model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Inisialisasi LabelEncoder
le = LabelEncoder()

# Fit dan transform label menjadi nilai numerik
train_labels = le.fit_transform(y_train)
valid_labels = le.transform(y_valid)

# Konversi data teks dan label menjadi dataset TensorFlow
train_ds = tf.data.Dataset.from_tensor_slices((X_train['FullText'], train_labels)).batch(32)
valid_ds = tf.data.Dataset.from_tensor_slices((X_valid['FullText'], valid_labels)).batch(32)

# Latih model
history = model.fit(train_ds, validation_data=valid_ds, epochs=15)

Epoch 1/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 28s 341ms/step - accuracy: 0.1538 - loss: 2.9310 - val_accuracy: 0.2575 - val_loss: 2.4529
Epoch 2/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 320ms/step - accuracy: 0.2893 - loss: 2.1215 - val_accuracy: 0.3575 - val_loss: 2.2657
Epoch 3/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 323ms/step - accuracy: 0.5690 - loss: 1.4739 - val_accuracy: 0.4600 - val_loss: 1.8648
Epoch 4/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 317ms/step - accuracy: 0.6832 - loss: 1.0927 - val_accuracy: 0.4175 - val_loss: 2.0082
Epoch 5/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 316ms/step - accuracy: 0.7331 - loss: 0.8615 - val_accuracy: 0.4550 - val_loss: 2.0959
Epoch 6/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 314ms/step - accuracy: 0.7789 - loss: 0.7134 - val_accuracy: 0.5200 - val_loss: 2.0066
Epoch 7/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 317ms/step - accuracy: 0.8479 - loss: 0.5283 - val_accuracy: 0.5375 - val_loss: 2.0350
Epoch 8/15
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 317ms/step - accuracy: 0.8888 - loss: 0.3719 - val_accu